In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Purpose: After setting up the cave client
token and access, purpose is to demonstrate
how to use CAVE to interface with microns data
- retrieve mesh
- retrieve, query synapses
- 

CAVE Documentation source:
https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/MeshAccess.ipynb
"""

'\nPurpose: After setting up the cave client\ntoken and access, purpose is to demonstrate\nhow to use CAVE to interface with microns data\n- retrieve mesh\n- retrieve, query synapses\n- \n\nCAVE Documentation source:\nhttps://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/mm3_intro/MeshAccess.ipynb\n'

In [3]:
# !pip3 install cloud-volume --upgrade
# !pip3 install caveclient --upgrade
# !pip3 install load_dotenv

In [4]:
import os
from dotenv import load_dotenv

import cloudvolume
import caveclient

In [5]:
cloudvolume.secrets.cave_credentials()

{'token': 'ba0a1b546e9b87f402c5d8c1d1d45bca'}

In [6]:
from neurd import cave_client_utils as ccu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
from neurd.cave_interface import CaveInterface
from pathlib import Path
client = CaveInterface(
    "minnie65_public",
    env_filepath=Path("./.env").absolute())
client

In [ ]:
cloudvolume.secrets.secretpath()

In [ ]:
/root/.cloudvolume/secrets
51776982dd531b1e1305fe173f851ad8

In [8]:
ccu.mesh_from_seg_id(
    seg_id = 864691137197197121,
    client=client,
)

Mesh(vertices<4237977>, faces<8460471>, normals<0>, segid=864691137197197121, encoding_type=<draco>)

In [11]:
from pathlib import Path
from python_tools import system_utils as su
from python_tools import json_utils as jsu

data = {"token": "ba0a1b546e9b87f402c5d8c1d1dbcelii"}

cloudvolume_secrets_path = Path("/root/.cloudvolume/secrets")
cloudvolume_secrets_path.mkdir(exist_ok = True)
json_file = jsu.dict_to_json_file(
    data,
    cloudvolume_secrets_path / Path("cave-secret.json")
)
json_file

In [ ]:
authorization_token = "ba0a1b546e9b87f402c5d8c1d1d45bca"

In [ ]:
~/.cloudvolume/secrets/cave-client.json

In [ ]:
For Google Storage (setup instructions here), default account credentials will be used if available and no service account is provided.

In [ ]:
debug

In [ ]:
"""
Retrieved the token and put in a local env file
"""

env_filepath = "./.env"
load_dotenv(env_filepath)
cave_token = os.getenv('CAVE_TOKEN')


In [ ]:
# Setting the release version for data
from caveclient import CAVEclient
client = CAVEclient('minnie65_public_v117')
client = CAVEclient('minnie65_public')
#client = CAVEclient('microns_public_v117')
client.auth.token = cave_token

In [ ]:
# this shows you the basic information about this datastack within CAVE
client.info.get_datastack_info()

# Utility Functions

In [ ]:
neuron_non_neuron_table = 'nucleus_neuron_svm'
synapse_table = 'synapses_pni_2'
nuclues_table = "nucleus_detection_v0"
release_table = "proofreading_status_public_release"

In [ ]:
# here you can see how many entries are in a table
def table_size(table_name):
    return client.materialize.get_annotation_count(
        table_name
    )

table_size(synapse_table)

In [ ]:
import numpy as np

def release_info():
    return  client.materialize.get_table_metadata(
        release_table
    )

def voxel_to_nm_scaling():
    return np.array(
        release_info()['voxel_resolution']
    )

voxel_to_nm_scaling()

In [ ]:
def neuron_nucleus_df(verbose = False):
    nuc_df = client.materialize.query_table(
        neuron_non_neuron_table,
        filter_equal_dict={'cell_type':'neuron'},
        filter_out_dict={'pt_root_id':[0]}
    ) 
    
    if verbose:
        print(f"# of nuclei = {len(verbose)}")
    
    return nuc_df

def segment_ids_with_nucleus(verbose=False):
    return_list = np.unique(nuc_df["pt_root_id"].to_numpy())
    if verbose:
        print(f"# of segments with at least one nucleus: {len(return_list)}")
    
    return return_list

segment_ids_with_nucleus(verbose = True)

# Getting Only Neurons

In [ ]:


nuc_df.head()

# Getting synapses

In [ ]:
# pick a segment you selected from neuroglancer to enter here
seg_id = 864691135474648896
seg_id = 864691137197197121

# inputs/outputs
output_df = client.materialize.synapse_query(pre_ids=seg_id)
input_df = client.materialize.synapse_query(post_ids=seg_id)
len(output_df), len(input_df)
output_df.head()

# Downloading a Mesh

In [ ]:
cloudvolume.CloudVolume?

In [ ]:
# to access the minnie65 public release dataset
# you initialize the client like this
cv = cloudvolume.CloudVolume(
    client.info.segmentation_source(),
    progress=False,
    use_https = True
)
# which, given a root_id, can be used to get a mesh
# cloud volume returns a dictionary with the neuron segment id as the key 
# and the mesh as the value
example_cell_id = 864691137197197121
mesh = cv.mesh.get(example_cell_id)[example_cell_id]

In [ ]:
# as a test, lets see that we can view the tables we can query
# from the materialization engine
client.materialize.get_tables()

In [ ]:
# this can be used to initialize a cloudvolume object
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False)
example_cell_id = 864691135474648896
mesh = cv.mesh.get(example_cell_id)[example_cell_id]